# 05 — Scenario Analysis & ROI

Translate forecast improvements into business impact.

## Business framing

In [17]:
import pandas as pd
import numpy as np


## Assume lead time & inventory

In [18]:
df = pd.read_parquet("../data/processed/features.parquet")

df = df.sort_values(["id", "date"]).reset_index(drop=True)

df.head()


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,snap_WI,sell_price,dow,week,lag_1,lag_7,lag_14,lag_28,rmean_7,rmean_28
0,FOODS_1_003_TX_3_validation,FOODS_1_003,FOODS_1,FOODS,TX_3,TX,d_1577,0,2015-05-24,11517,...,0,3.23,6,21,0.0,0.0,0.0,0.0,0.0,0.035714
1,FOODS_1_003_TX_3_validation,FOODS_1_003,FOODS_1,FOODS,TX_3,TX,d_1578,0,2015-05-25,11517,...,0,3.23,0,22,0.0,0.0,0.0,0.0,0.0,0.035714
2,FOODS_1_003_TX_3_validation,FOODS_1_003,FOODS_1,FOODS,TX_3,TX,d_1579,0,2015-05-26,11517,...,0,3.23,1,22,0.0,0.0,0.0,0.0,0.0,0.035714
3,FOODS_1_003_TX_3_validation,FOODS_1_003,FOODS_1,FOODS,TX_3,TX,d_1580,0,2015-05-27,11517,...,0,3.23,2,22,0.0,0.0,0.0,0.0,0.0,0.035714
4,FOODS_1_003_TX_3_validation,FOODS_1_003,FOODS_1,FOODS,TX_3,TX,d_1581,0,2015-05-28,11517,...,0,3.23,3,22,0.0,0.0,0.0,0.0,0.0,0.035714


In [19]:
HORIZON = 28


In [20]:
last_train_date = df["date"].max() - pd.Timedelta(days=HORIZON)

train = df[df["date"] <= last_train_date].copy()
future = df[df["date"] > last_train_date].copy()

train["date"].max(), future["date"].min(), future["date"].max()


(Timestamp('2016-03-27 00:00:00'),
 Timestamp('2016-03-28 00:00:00'),
 Timestamp('2016-04-24 00:00:00'))

In [21]:
categorical_cols = [
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "weekday",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2"
]

numeric_cols = [
    c for c in train.columns
    if c not in categorical_cols + ["id", "date", "sales", "d"]
]

features = categorical_cols + numeric_cols


In [22]:
for col in categorical_cols:
    train[col] = train[col].astype("category")
    future[col] = future[col].astype("category")


In [23]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(
    num_leaves=31,
    max_depth=10,
    learning_rate=0.05,
    n_estimators=500,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    train[features],
    train["sales"],
    categorical_feature=categorical_cols
)


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049071 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2005
[LightGBM] [Info] Number of data points in the train set: 307758, number of used features: 24
[LightGBM] [Info] Start training from score 1.174433
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,10
,learning_rate,0.05
,n_estimators,500
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [ ]:
future["forecast"] = model.predict(future[features])

In [ ]:
future["lead_time_days"] = 7
future["current_inventory"] = 50

## Aggregate demand over lead time

In [25]:
order_plan = (
    future.groupby("id")
    .agg(
        forecast_7d=("forecast", "sum"),
        inventory=("current_inventory", "mean")
    )
)

order_plan.head()


,forecast_7d,inventory
id,,
FOODS_1_003_TX_3_validation,4.249337,50.0
FOODS_1_005_CA_1_validation,39.382883,50.0
FOODS_1_005_TX_1_validation,31.894856,50.0
FOODS_1_005_TX_3_validation,39.248171,50.0
FOODS_1_008_CA_2_validation,12.201946,50.0


## Safety stock & order quantity

In [26]:
order_plan["safety_stock"] = 0.2 * order_plan["forecast_7d"]

order_plan["recommended_order"] = (
    order_plan["forecast_7d"]
    + order_plan["safety_stock"]
    - order_plan["inventory"]
).clip(lower=0)

order_plan.head()


,forecast_7d,inventory,safety_stock,recommended_order
id,,,,
FOODS_1_003_TX_3_validation,4.249337,50.0,0.849867,0.0
FOODS_1_005_CA_1_validation,39.382883,50.0,7.876577,0.0
FOODS_1_005_TX_1_validation,31.894856,50.0,6.378971,0.0
FOODS_1_005_TX_3_validation,39.248171,50.0,7.849634,0.0
FOODS_1_008_CA_2_validation,12.201946,50.0,2.440389,0.0


## ROI estimation

In [33]:
unit_cost = 30
inventory_reduction_pct = 0.17

annual_savings = (
    order_plan["safety_stock"].sum()
    * unit_cost
    * inventory_reduction_pct
)

annual_savings


np.float64(35505.476457161625)

## Final takeaway

In [36]:
print(
    f"Estimated annual inventory savings (pilot scale): "
    f"₹{annual_savings:,.0f}"
)


Estimated annual inventory savings (pilot scale): ₹35,505
